In [1]:
! pip install -q langchain-core langchain-community langchain-google-genai beautifulsoup4 faiss-cpu

In [2]:
from google.colab import userdata
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage,AIMessage

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
LANGCHAIN_PROJECT = userdata.get('LANGCHAIN_PROJECT')

In [4]:
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['LANGCHAIN_PROJECT'] = LANGCHAIN_PROJECT
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [5]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [6]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [7]:
model.invoke("Hi").content

'Hi there! How can I help you today?'

In [8]:
loader = WebBaseLoader(web_paths=("https://en.wikipedia.org/wiki/Artificial_intelligence",),bs_kwargs={"parse_only":bs4.SoupStrainer("p"),})

In [9]:
doc = loader.load()

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = splitter.split_documents(doc)

In [11]:
vector_store = FAISS.from_documents(chunks, embedding)
retriever = vector_store.as_retriever()

In [12]:
system_prompt = '''
You are helpful assistant which can answer any queston related to artificial intelligense
{context}
'''

In [13]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("user","{input}")
    ]
)

In [14]:
question_answering_chain = create_stuff_documents_chain(model,chat_prompt)

In [15]:
rag_chain = create_retrieval_chain(retriever,question_answering_chain)

In [16]:
response = rag_chain.invoke({"input":"What is artificial intelligence?"})

In [17]:
response["answer"]

"Artificial intelligence (AI), in its broadest sense, is intelligence demonstrated by machines, particularly computer systems.  It's a field of computer science focused on developing methods and software that allow machines to perceive their environment, learn from experience, and take actions to achieve defined goals.  These machines, capable of exhibiting AI, are often referred to as AIs."

In [18]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [19]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
     ]
)

In [20]:
history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)

In [21]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [22]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)


In [23]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [24]:
chat_history = []

In [25]:
question="What is artificial intelligence"

In [26]:
response1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
response1["answer"]

"Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems.  It's a field of computer science focused on developing methods and software that allow machines to perceive their environment, learn from experience, and take actions to achieve defined goals.  This involves creating systems that can mimic human cognitive functions like learning, problem-solving, and decision-making.  While the definition can be broad, the core idea is creating machines capable of intelligent behavior."

In [27]:
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"]),
    ]
)

In [28]:
chat_history

[HumanMessage(content='What is artificial intelligence', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems.  It's a field of computer science focused on developing methods and software that allow machines to perceive their environment, learn from experience, and take actions to achieve defined goals.  This involves creating systems that can mimic human cognitive functions like learning, problem-solving, and decision-making.  While the definition can be broad, the core idea is creating machines capable of intelligent behavior.", additional_kwargs={}, response_metadata={})]

In [29]:
question2 = "types of artificial intelligence"

In [30]:
response2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})
response2["answer"]

"There are several ways to categorize types of artificial intelligence, and the lines between them can be blurry.  Here are a few common classifications:\n\n**Based on Capability:**\n\n* **Narrow or Weak AI:** This is the most common type of AI today.  It's designed to perform a specific task, like playing chess, recommending products, or recognizing faces.  It excels at its designated task but lacks general intelligence or the ability to adapt to new situations outside its programming.  Examples include Siri, Alexa, and spam filters.\n\n* **General or Strong AI:** This hypothetical type of AI possesses human-level intelligence and can perform any intellectual task that a human being can.  It would be capable of learning, reasoning, problem-solving, and adapting to new situations across a wide range of domains.  General AI does not currently exist.\n\n* **Super AI:** This is a hypothetical AI that surpasses human intelligence in all aspects.  It's a concept often explored in science fi

In [31]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [32]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [33]:
conversational_rag_chain.invoke(
    {"input": "applications of ai"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'AI applications are incredibly diverse and span numerous fields.  Here\'s a breakdown categorized for clarity:\n\n**High-Profile & Widely Used Applications:**\n\n* **Search Engines:**  Powering advanced search capabilities like Google Search, understanding natural language queries and delivering relevant results.\n* **Recommendation Systems:**  Used by platforms like Amazon, Netflix, and YouTube to suggest products, movies, and videos based on user preferences and behavior.\n* **Virtual Assistants:**  Examples include Siri, Alexa, and Google Assistant, providing voice-activated assistance for tasks like scheduling, information retrieval, and controlling smart home devices.\n* **Autonomous Vehicles:**  Self-driving cars and trucks utilize AI for navigation, object detection, and decision-making.  Companies like Waymo are at the forefront of this technology.\n* **Generative AI:** Tools like ChatGPT, DALL-E 2, and Midjourney create text, images, videos, and other data in response to prom

In [34]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='applications of ai', additional_kwargs={}, response_metadata={}), AIMessage(content='AI applications are incredibly diverse and span numerous fields.  Here\'s a breakdown categorized for clarity:\n\n**High-Profile & Widely Used Applications:**\n\n* **Search Engines:**  Powering advanced search capabilities like Google Search, understanding natural language queries and delivering relevant results.\n* **Recommendation Systems:**  Used by platforms like Amazon, Netflix, and YouTube to suggest products, movies, and videos based on user preferences and behavior.\n* **Virtual Assistants:**  Examples include Siri, Alexa, and Google Assistant, providing voice-activated assistance for tasks like scheduling, information retrieval, and controlling smart home devices.\n* **Autonomous Vehicles:**  Self-driving cars and trucks utilize AI for navigation, object detection, and decision-making.  Companies like Waymo are at the forefr

In [35]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]


'The "doing it" refers to building and implementing AI applications.  There\'s no single "way," but rather a collection of techniques and approaches depending on the specific application and goals.  Here are some common methods:\n\n**1. Data Acquisition and Preparation:**\n\n* **Data Collection:** Gathering relevant data is the crucial first step. This might involve scraping web data, using APIs, collecting sensor data, or utilizing existing datasets. The quality and quantity of data directly impact the AI model\'s performance.\n* **Data Cleaning:**  Raw data is rarely perfect. Cleaning involves handling missing values, removing outliers, and correcting inconsistencies.\n* **Data Preprocessing:**  This step transforms the data into a format suitable for the chosen AI model. Techniques include normalization, standardization, feature scaling, and encoding categorical variables.\n* **Data Augmentation:**  Increasing the size of the dataset by creating modified versions of existing data po

In [36]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: applications of ai

AI: AI applications are incredibly diverse and span numerous fields.  Here's a breakdown categorized for clarity:

**High-Profile & Widely Used Applications:**

* **Search Engines:**  Powering advanced search capabilities like Google Search, understanding natural language queries and delivering relevant results.
* **Recommendation Systems:**  Used by platforms like Amazon, Netflix, and YouTube to suggest products, movies, and videos based on user preferences and behavior.
* **Virtual Assistants:**  Examples include Siri, Alexa, and Google Assistant, providing voice-activated assistance for tasks like scheduling, information retrieval, and controlling smart home devices.
* **Autonomous Vehicles:**  Self-driving cars and trucks utilize AI for navigation, object detection, and decision-making.  Companies like Waymo are at the forefront of this technology.
* **Generative AI:** Tools like ChatGPT, DALL-E 2, and Midjourney create text, images, videos, and other data